In [1]:
import torch
from torch import nn
import lightnet as ln
from torchinfo import summary

/home/gustavo/workstation/depth_estimation/codes/RGBD-fusion-detection/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = ln.models.Darknet19(1000)
model.load('weights/darknet19_448.weights')

# Save as PyTorch weight file (Not strictly necessary, but it is faster than darknet weight files)
model.save('weights/darknet19_448.pt')

# Converting Darknet19 weights to Yolo (This is the same as the darknet19_448.conv.23.weights from darknet)
model.save('weights/yolo-pretrained_darknet.pt', remap=ln.models.YoloV2.remap_darknet19)

# Load yolo weights (Requires `strict=False`, because not all layers have weights in this file)
detection_model = ln.models.YoloV2(3)
detection_model.load('weights/yolo-pretrained_darknet.pt', strict=False)

Weight file uses sizeof to compute variable size, which might lead to undefined behaviour. (choosing int=int32, float=float32)
/home/gustavo/workstation/depth_estimation/codes/RGBD-fusion-detection/.venv/lib/python3.10/site-packages/lightnet/network/module/_lightnet.py:293: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any u

In [8]:
summary(model=detection_model, 
        input_size=(1, 3, 416, 416), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
YoloV2 (YoloV2)                               [1, 3, 416, 416]     [1, 125, 13, 13]     --                   True
├─FeatureExtractor (backbone)                 [1, 3, 416, 416]     [1, 1024, 13, 13]    --                   True
│    └─Sequential (module)                    [1, 3, 416, 416]     [1, 1024, 13, 13]    --                   True
│    │    └─Conv2dBatchAct (1_convbatch)      [1, 3, 416, 416]     [1, 32, 416, 416]    928                  True
│    │    └─MaxPool2d (2_max)                 [1, 32, 416, 416]    [1, 32, 208, 208]    --                   --
│    │    └─Conv2dBatchAct (3_convbatch)      [1, 32, 208, 208]    [1, 64, 208, 208]    18,560               True
│    │    └─MaxPool2d (4_max)                 [1, 64, 208, 208]    [1, 64, 104, 104]    --                   --
│    │    └─Conv2dBatchAct (5_convbatch)      [1, 64, 104, 104]    [1, 128, 104, 104]  

In [ ]:
for i in model.state_dict().keys():
    print(i)

In [14]:
model_fusion = ln.models.YoloFusion(20)

In [15]:
summary(model=model_fusion, 
        input_size=(1, 3, 416, 416), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [16]:
for i in model_fusion.state_dict().keys():
    print(i)

layers.0.fuse.weight
layers.0.1_convbatch.layers.0.weight
layers.0.1_convbatch.layers.1.weight
layers.0.1_convbatch.layers.1.bias
layers.0.1_convbatch.layers.1.running_mean
layers.0.1_convbatch.layers.1.running_var
layers.0.1_convbatch.layers.1.num_batches_tracked
layers.1.combined.3_convbatch.layers.0.weight
layers.1.combined.3_convbatch.layers.1.weight
layers.1.combined.3_convbatch.layers.1.bias
layers.1.combined.3_convbatch.layers.1.running_mean
layers.1.combined.3_convbatch.layers.1.running_var
layers.1.combined.3_convbatch.layers.1.num_batches_tracked
layers.1.combined.5_convbatch.layers.0.weight
layers.1.combined.5_convbatch.layers.1.weight
layers.1.combined.5_convbatch.layers.1.bias
layers.1.combined.5_convbatch.layers.1.running_mean
layers.1.combined.5_convbatch.layers.1.running_var
layers.1.combined.5_convbatch.layers.1.num_batches_tracked
layers.1.combined.6_convbatch.layers.0.weight
layers.1.combined.6_convbatch.layers.1.weight
layers.1.combined.6_convbatch.layers.1.bias
lay